In [1]:
import os
import cv2
import json
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pycocotools.coco import COCO

In [2]:
def split_video_into_reps(video_path, json_path, save_path):
    coco = COCO(json_path)
    cap = cv2.VideoCapture(video_path)
    FPS = int(cap.get(cv2.CAP_PROP_FPS))
    FRAME_W = int(cap.get(3)) 
    FRAME_H = int(cap.get(4))

    img_iter = iter(coco.imgs.values())
    ann_iter = iter(coco.anns.values())

    json_data = dict()
    json_data['info'] = coco.dataset['info']
    json_data['categories'] = coco.dataset['categories']
    imgs = []
    anns = []
    rep_count = 0
    video_writer = cv2.VideoWriter(f"{save_path}_{rep_count}.mp4", cv2.VideoWriter_fourcc(*'MP4V'), 
                                   FPS, (FRAME_W, FRAME_H))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img_data = next(img_iter)
        if img_data['rep_count'] > rep_count + 1:
            json_data['images'] = imgs
            json_data['annotations'] = anns
            with open(f"{save_path}_{rep_count}.json", 'w', encoding='utf-8') as f:
                json.dump(json_data, f, ensure_ascii=False, indent=4)

            rep_count += 1
            video_writer = cv2.VideoWriter(f"{save_path}_{rep_count}.mp4", cv2.VideoWriter_fourcc(*'MP4V'), 
                                           FPS, (FRAME_W, FRAME_H))
            imgs = []
            anns = []

        video_writer.write(frame) 
        imgs.append(img_data)
        anns.append(next(ann_iter))

    rep_count += 1
    json_data['images'] = imgs
    json_data['annotations'] = anns
    with open(f"{save_path}_{rep_count}.json", 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

    video_writer = cv2.VideoWriter(f"{save_path}_{rep_count}.mp4", cv2.VideoWriter_fourcc(*'MP4V'), 
                                   FPS, (FRAME_W, FRAME_H))

    cap.release()
    video_writer.release()

In [ ]:
og_data_folder = 'InfinityRepData/data'
new_data_folder = 'InfinityRepData/rep_data'

og_data_files = os.listdir(og_data_folder)
for i in range(1, len(og_data_files), 3):
    file_basename = og_data_files[i].split('.')[0]
    video_path = os.path.join(og_data_folder, f"{file_basename}.mp4")
    json_path = os.path.join(og_data_folder, f"{file_basename}.json")
    save_path = os.path.join(new_data_folder, file_basename)

    split_video_into_reps(video_path, json_path, save_path)

In [ ]:
# For reference: https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py
# TODO: What to do with the .zip file of labels per video? Currently ignoring